# PiNN as a potential  

For both ice and liquid water trained on ab-initial data.

## Introduction

One of the primiary goal of PiNN is to serve as a verstile force field.

Since the training part is rather costly (around a day with a descent GPU),
we've prepared a pre-trained model for you if you just want to try it out.

## Using the pre-trained model

PiNN predicts energies and forces given a atomic structure. To use PiNN 
as a potential, you need a molucular dynamics driver that updates atomic 
positions given the forces. Here we use ASE as a driver.

In [ ]:
from ase import units
from ase.io import read
from ase.io.trajectory import Trajectory
from ase.md.nptberendsen import NPTBerendsen
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

from pinn.models import potential_model
from pinn.calculator import PiNN_calc

### Setting up the calculator

In [ ]:
model = potential_model('blyp_water_bpnn')
calc = PiNN_calc(model)
atoms = read('waterbox.xyz')
atoms.set_calculator(calc)

### Setting up the molecular dynamics calcualtion

In [ ]:
MaxwellBoltzmannDistribution(atoms, 300.*units.kB)
dt = 0.5 * units.fs
dyn = NPTBerendsen(atoms, timestep=dt, temperature=330, pressure=1,
                  taut=dt * 100, taup=dt * 1000, compressibility=4.57e-5)
traj = Trajectory('npt.traj', 'w', atoms)
dyn.attach(traj.write)
dyn.run(1000)

# Visualize the trajectory 

To run this you need [NGLView](https://github.com/arose/nglview)

In [ ]:
import nglview as nv

traj = read('npt.traj', index=':')
v = nv.show_asetraj(traj)
v.clear_representations()
v.add_spacefill(radius_scale=0.5)
v.add_unitcell()
v